In [1]:
%reset -fs

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

import re



In [2]:
df = pd.read_csv('eu_tweet_df_eda1.csv')

/var/folders/gc/kgfpjt4x1d5c1thdk2sl_2hr0000gn/T/ipykernel_19458/3349357611.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('eu_tweet_df_eda1.csv')


In [3]:
def strip_punc_lower_http(txt):
    txt = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", txt)

    txt = re.sub(pattern= "[^\w\s]",
            repl="",
            string=txt).lower()
    
    txt = re.sub(pattern= "[0-9]",
            repl="",
            string=txt)
    
    return txt
strip_punc_lower_http('I am, testing #ALEX > than X 10. https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html')

'i am testing alex  than x   '

In [4]:
df['text'] = df['text'].apply(lambda x: strip_punc_lower_http(x))

In [5]:
df.head()

,id,created_at,text,user_name,user_screen_name,user_id,user_description,user_location,user_verified,lang,...,in_reply_to_user_id,retweet_count,retweet_id,retweet_screen_name,user_created_at,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
0,866239077045207040,Sun May 21 10:27:44 +0000 2017,pounding the streets of redland this morning w...,Green💚🌻Molly,GreenPartyMolly,726372601,Molly Scott Cato 👋 Green Party External Comms ...,South West,False,en,...,NaN,23.0,8.662375e+17,bristol_yg,Mon Jul 30 16:21:34 +0000 2012,27154.0,59652.0,1965.0,854.0,69671.0
1,866232717196840960,Sun May 21 10:02:28 +0000 2017,the ukip battle bus is coming please support m...,Dr Julia Reid PhD,julia_reid,43554444,Former Brexit Party MEP for the SW (2014-19) -...,Calne,True,en,...,NaN,38.0,8.662321e+17,DrTeckKhong,Sat May 30 16:46:20 +0000 2009,62208.0,4773.0,6282.0,242.0,139670.0
2,866236815463862272,Sun May 21 10:18:45 +0000 2017,yesterday was th european maritime day today i...,Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,NaN,21.0,8.662204e+17,EU_MARE,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0
3,866236868404314112,Sun May 21 10:18:57 +0000 2017,thesnp depute leader and candidate for moray a...,Ian Hudghton,HudghtonSNP,308677137,SNP President 2005 - 2020. SNP Member of the E...,Scotland,False,en,...,NaN,6.0,8.662227e+17,erikgeddes,Tue May 31 20:22:38 +0000 2011,21.0,5044.0,398.0,205.0,1789.0
4,866233710714257408,Sun May 21 10:06:25 +0000 2017,cacophonic that and fox hunting and dementia t...,Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,25067514.0,0.0,NaN,NaN,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0


In [6]:
df_last500k = df.loc[len(df)-500000:,['id', 'created_at', 'text',
                                      'user_description','user_location']].copy()

In [7]:
df_last500k

,id,created_at,text,user_description,user_location
2689504,1011761027283935232,Wed Jun 27 00:00:01 +0000 2018,very disappointed in the scotus decision to up...,Member of Congress representing the First Dist...,"Hartford, CT"
2689505,1011757594787155968,Tue Jun 26 23:46:22 +0000 2018,i love it blackfirebrigad opens new clubhouse ...,Official Twitter for Rep Robin Kelly #IL02\n\n...,NaN
2689506,1011759406059421696,Tue Jun 26 23:53:34 +0000 2018,a stronger economy means a stronger australia ...,Fmr Assistant Minister & Liberal MP | Dees Man...,"Brighton East, VIC"
2689507,1011756236281470976,Tue Jun 26 23:40:58 +0000 2018,thought the brexit dividend was paying for the...,"MEP '99-2019, Rtd NHS Consultant. Internationa...",UK & EU
2689508,1011757023011061760,Tue Jun 26 23:44:06 +0000 2018,bob zimmer understanding bill c and the new ru...,MP for Prince George-Peace River-Northern Rock...,"Fort St. John, British Columbia"
...,...,...,...,...,...
3189499,1125114865423613952,Sun May 05 19:07:42 +0000 2019,delighted to welcome borisjohnson to clwydwest...,Conservative MP for Clwyd West / Aelod Seneddo...,North Wales and Westminster
3189500,1124728667555467264,Sat May 04 17:33:05 +0000 2019,stopped in at the churchill united church for ...,Former IAFF (2727) Firefighter. Member of Parl...,"Barrie, Ontario"
3189501,1124391243885289472,Fri May 03 19:12:17 +0000 2019,proest,Beste Kamerlid (GeenStijl). Politicus van het ...,Bourgondisch Kreits
3189502,1124682517389426688,Sat May 04 14:29:42 +0000 2019,fantastic to join isha jp and niaz at their gr...,Former IAFF (2727) Firefighter. Member of Parl...,"Barrie, Ontario"


In [8]:
my_stop_words = ['wonderful', 'did', 'didnt', 'issue', 'issues', 'actually', 'id','th',
                'community']

In [9]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(my_stop_words)

In [10]:
tv_30docs = TfidfVectorizer(stop_words = my_stop_words, min_df=30)
dtm_tv_30docs = tv_30docs.fit_transform(list(df_last500k['text']))

In [11]:
dtm_tv_30docs.shape

(500000, 15895)

In [ ]:
nmf_topics = NMF(n_components=40, init='random', max_iter=1000)
nmf_17k_words = nmf_topics.fit_transform(dtm_tv_30docs)

In [ ]:
# Function to display the top n terms in each topic
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

In [ ]:
output_17kwords = display_topics(nmf_topics, tv_30docs.get_feature_names_out(), 10,
                                          topic_names=None)

In [ ]:
total = 0
for i in range(40):
    print(f'topic{i+1} {sum(nmf_topics.components_[i])}')
    total += sum(nmf_topics.components_[i])
total